In [1]:
import numpy as np
import random as rd
from docplex.mp.model import Model

In [2]:
def gen_instance_one_product(T : int, verbose = False):
    period_set =  range(T)
    alpha = float(round(rd.uniform(0.6, 0.9), 2))
    gamma = float(round(rd.uniform(0.2, 0.4), 2))
    set_up_cost = [rd.randint(0, 10) for _ in period_set]
    variable_prod_cost = [rd.randint(0, 10) for _ in period_set]
    inventory_cost = [rd.randint(0, 5) for _ in period_set]
    mtn_cost = [rd.randint(0, 100) for _ in period_set]
    demand = [rd.randint(0, 50) for _ in period_set]
    cmax = rd.randint(40, 50)
    if verbose: 
        print('Set of periods in the planning horizon T=' + str(T))
        print()
        print('Capacity reduction coefficient alpha = ' + str(alpha))
        print()
        print('Relative decrease of capacity per set up gamma = ' + str(gamma))
        print()
        print('Set_up cost in each period f = ' + str(np.array(set_up_cost)))
        print()
        print('Variable production cost per unit in each period p = ' + str(np.array(variable_prod_cost)))
        print()
        print('Inventory holding cost per unit by the end of each period h = ' + str(np.array(inventory_cost)))
        print()
        print('Demand in each period d = ' + str(np.array(demand)))
        print()
        print('Maintenance cost in each period m =  ' + str(mtn_cost))
        print()
        print('Maximum capacity of the machine cmax = ' + str(cmax))
        print()
    return {'T': T, 'alpha': alpha, 'gamma': gamma, 'set_up_cost': set_up_cost, 'variable_prod_cost': variable_prod_cost, 'inventory_cost': inventory_cost, 'demand': demand,
            'mtn_cost': mtn_cost, 'cmax': cmax}

In [3]:
T = 10
period_set = [i for i in range(T)]
instance_dict = gen_instance_one_product(T)
set_up_cost = instance_dict["set_up_cost"]
variable_prod_cost = instance_dict["variable_prod_cost"]
inventory_cost = instance_dict["inventory_cost"]
demand = instance_dict["demand"]
mtn_cost = instance_dict["mtn_cost"]
cmax = instance_dict["cmax"]
alpha = instance_dict["alpha"]
print(mtn_cost)

[38, 17, 63, 66, 99, 70, 14, 83, 82, 42]


In [48]:
C_t = np.array([[cmax for _ in period_set]])
Mc = [sum(mtn_cost)]
objectives = []
#master
master = Model(name="clspm")
#x = master.integer_var_matrix(period_set, period_set, name = "Lot variable")
x = {(i,j): master.continuous_var(name=f'x_{i}_{j}') for i in range(T) for j in range(i, T)} #lot variable

y = master.continuous_var_list(period_set, name = "y") #set_up variable

c = master.continuous_var_list(period_set,name = "c") #capacity variable

z_1 = master.continuous_var(name = "z_1") #maintenance variable


contraintes1 = master.add_constraints(master.sum(x[k,t] for k in range(t+1)) == demand[t] for t in period_set)
"""
# Ajouter les contraintes avec des noms personnalisés
for i, ct in enumerate(contraintes):
    master.add_constraint(ct, ctname=f"contrainte{i}")

# Afficher les noms des contraintes
for ct in master.iter_constraints():
    print(ct.name)
"""
contraintes2 = master.add_constraints(x[k,t] <= demand[t]*y[k] for k in period_set for t in range(k,T)) 

contraintes3 = master.add_constraints(master.sum(x[k,t] for t in range(k,T)) <= c[k] for k in period_set)

contrainte4 = master.add_constraint(z_1 == 1)

contraintes5 = master.add_constraints(c[t] == C_t[0,t]*z_1 for t in period_set)

#Fonction objectif
cost1 = sum(set_up_cost[k]*y[k] for k in period_set)
cost2 = sum(variable_prod_cost[k]*sum(x[k,t] for t in range(k,T)) for k in period_set)
cost3 = sum(x[k,t]*sum(inventory_cost[j] for j in range(k,t)) for k in period_set for t in range(k,T))
cost4 = Mc[0]*z_1
master.obj = cost1 + cost2 + cost3 + cost4
master.minimize(master.obj)

print(master.export_as_lp_string())

# Résolution
master_sol = master.solve(log_output = True)
objectives.append(master_sol.get_objective_value())
#master.print_information()
master.print_solution()


\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: clspm

Minimize
 obj: 7 x_0_0 + 11 x_0_1 + 11 x_0_2 + 16 x_0_3 + 16 x_0_4 + 17 x_0_5 + 18 x_0_6
      + 23 x_0_7 + 25 x_0_8 + 25 x_0_9 + 4 x_1_1 + 4 x_1_2 + 9 x_1_3 + 9 x_1_4
      + 10 x_1_5 + 11 x_1_6 + 16 x_1_7 + 18 x_1_8 + 18 x_1_9 + 7 x_2_2
      + 12 x_2_3 + 12 x_2_4 + 13 x_2_5 + 14 x_2_6 + 19 x_2_7 + 21 x_2_8
      + 21 x_2_9 + x_3_5 + 2 x_3_6 + 7 x_3_7 + 9 x_3_8 + 9 x_3_9 + 3 x_4_4
      + 4 x_4_5 + 5 x_4_6 + 10 x_4_7 + 12 x_4_8 + 12 x_4_9 + 9 x_5_5 + 10 x_5_6
      + 15 x_5_7 + 17 x_5_8 + 17 x_5_9 + 9 x_6_6 + 14 x_6_7 + 16 x_6_8
      + 16 x_6_9 + 9 x_7_7 + 11 x_7_8 + 11 x_7_9 + 2 x_8_8 + 2 x_8_9 + x_9_9
      + 9 y_1 + 4 y_2 + 8 y_3 + 4 y_4 + 6 y_5 + 10 y_6 + 9 y_8 + 7 y_9 + 574 z_1
Subject To
 c1: x_0_0 = 6
 c2: x_0_1 + x_1_1 = 6
 c3: x_0_2 + x_1_2 + x_2_2 = 48
 c4: x_0_3 + x_1_3 + x_2_3 + x_3_3 = 5
 c5: x_0_4 + x_1_4 + x_2_4 + x_3_4 + x_4_4 = 15
 c6: x_0_5 + x_1_5 + x_2_5 + x_3_5 + x_4_5 + x_5_5 

In [49]:
pi = master.dual_values(contraintes5)
pi1 = contrainte4.dual_value
def SP(pi: list, pi1: int):
    pricing = Model(name = "sous_probleme")
    z = pricing.binary_var_list(period_set, name = "z")
    c = pricing.integer_var_list(period_set, name = 'c')

    obj = sum(mtn_cost[t]*z[t] + pi[t]*c[t] for t in period_set) - pi1
    pricing.minimize(obj)

    ct1 = pricing.add_constraints(c[t] <= cmax for t in period_set)
    ct2 = pricing.add_constraints(c[t] <= alpha*c[t-1] + cmax*z[t] for t in period_set)

    # Résolution
    pricing_sol = pricing.solve(log_output = True)
    #master.print_information()
    pricing.print_solution()
    sc = np.array(pricing_sol.get_values(c))
    sz = pricing_sol.get_values(z)
    return sc, sz
    
sc, sz = SP(pi, pi1)

Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 381.472418 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 10 rows and 0 columns.
MIP Presolve modified 10 coefficients.
Reduced MIP has 10 rows, 20 columns, and 30 nonzeros.
Reduced MIP has 10 binaries, 10 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 10 rows, 20 columns, and 30 nonzeros.
Reduced MIP has 10 binaries, 10 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left    

In [50]:
Mc.append(sum(mtn_cost[t]*sz[t] for t in period_set))
C_t = np.vstack([C_t, sc])

z_2 = master.continuous_var(name = "z_2")

contrainte4.left_expr += z_2

for t in period_set:
    contraintes5[t].right_expr += sc[t]*z_2
    #print(contraintes5[t])

#print(Mc)
master.obj += Mc[1]*z_2
master.minimize(master.obj)

print(master.export_as_lp_string())
# Résolution
master_sol = master.solve(log_output = True)
objectives.append(master_sol.get_objective_value())
#master.print_information()
master.print_solution()


\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: clspm

Minimize
 obj: 7 x_0_0 + 11 x_0_1 + 11 x_0_2 + 16 x_0_3 + 16 x_0_4 + 17 x_0_5 + 18 x_0_6
      + 23 x_0_7 + 25 x_0_8 + 25 x_0_9 + 4 x_1_1 + 4 x_1_2 + 9 x_1_3 + 9 x_1_4
      + 10 x_1_5 + 11 x_1_6 + 16 x_1_7 + 18 x_1_8 + 18 x_1_9 + 7 x_2_2
      + 12 x_2_3 + 12 x_2_4 + 13 x_2_5 + 14 x_2_6 + 19 x_2_7 + 21 x_2_8
      + 21 x_2_9 + x_3_5 + 2 x_3_6 + 7 x_3_7 + 9 x_3_8 + 9 x_3_9 + 3 x_4_4
      + 4 x_4_5 + 5 x_4_6 + 10 x_4_7 + 12 x_4_8 + 12 x_4_9 + 9 x_5_5 + 10 x_5_6
      + 15 x_5_7 + 17 x_5_8 + 17 x_5_9 + 9 x_6_6 + 14 x_6_7 + 16 x_6_8
      + 16 x_6_9 + 9 x_7_7 + 11 x_7_8 + 11 x_7_9 + 2 x_8_8 + 2 x_8_9 + x_9_9
      + 9 y_1 + 4 y_2 + 8 y_3 + 4 y_4 + 6 y_5 + 10 y_6 + 9 y_8 + 7 y_9 + 574 z_1
      + 165 z_2
Subject To
 c1: x_0_0 = 6
 c2: x_0_1 + x_1_1 = 6
 c3: x_0_2 + x_1_2 + x_2_2 = 48
 c4: x_0_3 + x_1_3 + x_2_3 + x_3_3 = 5
 c5: x_0_4 + x_1_4 + x_2_4 + x_3_4 + x_4_4 = 15
 c6: x_0_5 + x_1_5 + x_2_5 + x_3_5 

In [51]:
pi = master.dual_values(contraintes5)
pi1 = contrainte4.dual_value
sc, sz = SP(pi, pi1)

Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 1235.105077 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 10 rows and 0 columns.
MIP Presolve modified 10 coefficients.
Reduced MIP has 10 rows, 20 columns, and 30 nonzeros.
Reduced MIP has 10 binaries, 10 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 10 rows, 20 columns, and 30 nonzeros.
Reduced MIP has 10 binaries, 10 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left   

In [52]:

Mc.append(sum(mtn_cost[t]*sz[t] for t in period_set))
C_t = np.vstack([C_t, sc])

z_3 = master.continuous_var(name = "z_3")

contrainte4.left_expr += z_3

for t in period_set:
    contraintes5[t].right_expr += sc[t]*z_3
    #print(contraintes5[t])

#print(Mc)
master.obj += Mc[2]*z_3
master.minimize(master.obj)

print(master.export_as_lp_string())
# Résolution
master_sol = master.solve(log_output = True)
objectives.append(master_sol.get_objective_value())
#master.print_information()
master.print_solution()

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: clspm

Minimize
 obj: 7 x_0_0 + 11 x_0_1 + 11 x_0_2 + 16 x_0_3 + 16 x_0_4 + 17 x_0_5 + 18 x_0_6
      + 23 x_0_7 + 25 x_0_8 + 25 x_0_9 + 4 x_1_1 + 4 x_1_2 + 9 x_1_3 + 9 x_1_4
      + 10 x_1_5 + 11 x_1_6 + 16 x_1_7 + 18 x_1_8 + 18 x_1_9 + 7 x_2_2
      + 12 x_2_3 + 12 x_2_4 + 13 x_2_5 + 14 x_2_6 + 19 x_2_7 + 21 x_2_8
      + 21 x_2_9 + x_3_5 + 2 x_3_6 + 7 x_3_7 + 9 x_3_8 + 9 x_3_9 + 3 x_4_4
      + 4 x_4_5 + 5 x_4_6 + 10 x_4_7 + 12 x_4_8 + 12 x_4_9 + 9 x_5_5 + 10 x_5_6
      + 15 x_5_7 + 17 x_5_8 + 17 x_5_9 + 9 x_6_6 + 14 x_6_7 + 16 x_6_8
      + 16 x_6_9 + 9 x_7_7 + 11 x_7_8 + 11 x_7_9 + 2 x_8_8 + 2 x_8_9 + x_9_9
      + 9 y_1 + 4 y_2 + 8 y_3 + 4 y_4 + 6 y_5 + 10 y_6 + 9 y_8 + 7 y_9 + 574 z_1
      + 165 z_2 + 221 z_3
Subject To
 c1: x_0_0 = 6
 c2: x_0_1 + x_1_1 = 6
 c3: x_0_2 + x_1_2 + x_2_2 = 48
 c4: x_0_3 + x_1_3 + x_2_3 + x_3_3 = 5
 c5: x_0_4 + x_1_4 + x_2_4 + x_3_4 + x_4_4 = 15
 c6: x_0_5 + x_1_5 + x_2_

In [53]:
pi = master.dual_values(contraintes5)
pi1 = contrainte4.dual_value
sc, sz = SP(pi, pi1)

Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 821.055751 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 10 rows and 0 columns.
MIP Presolve modified 10 coefficients.
Reduced MIP has 10 rows, 20 columns, and 30 nonzeros.
Reduced MIP has 10 binaries, 10 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 10 rows, 20 columns, and 30 nonzeros.
Reduced MIP has 10 binaries, 10 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left    

In [54]:
Mc.append(sum(mtn_cost[t]*sz[t] for t in period_set))
C_t = np.vstack([C_t, sc])

z_4 = master.continuous_var(name = "z_4")

contrainte4.left_expr += z_4

for t in period_set:
    contraintes5[t].right_expr += sc[t]*z_4
    #print(contraintes5[t])

#print(Mc)
master.obj += Mc[3]*z_4
master.minimize(master.obj)

print(master.export_as_lp_string())
# Résolution
master_sol = master.solve(log_output = True)
objectives.append(master_sol.get_objective_value())
#master.print_information()
master.print_solution()

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: clspm

Minimize
 obj: 7 x_0_0 + 11 x_0_1 + 11 x_0_2 + 16 x_0_3 + 16 x_0_4 + 17 x_0_5 + 18 x_0_6
      + 23 x_0_7 + 25 x_0_8 + 25 x_0_9 + 4 x_1_1 + 4 x_1_2 + 9 x_1_3 + 9 x_1_4
      + 10 x_1_5 + 11 x_1_6 + 16 x_1_7 + 18 x_1_8 + 18 x_1_9 + 7 x_2_2
      + 12 x_2_3 + 12 x_2_4 + 13 x_2_5 + 14 x_2_6 + 19 x_2_7 + 21 x_2_8
      + 21 x_2_9 + x_3_5 + 2 x_3_6 + 7 x_3_7 + 9 x_3_8 + 9 x_3_9 + 3 x_4_4
      + 4 x_4_5 + 5 x_4_6 + 10 x_4_7 + 12 x_4_8 + 12 x_4_9 + 9 x_5_5 + 10 x_5_6
      + 15 x_5_7 + 17 x_5_8 + 17 x_5_9 + 9 x_6_6 + 14 x_6_7 + 16 x_6_8
      + 16 x_6_9 + 9 x_7_7 + 11 x_7_8 + 11 x_7_9 + 2 x_8_8 + 2 x_8_9 + x_9_9
      + 9 y_1 + 4 y_2 + 8 y_3 + 4 y_4 + 6 y_5 + 10 y_6 + 9 y_8 + 7 y_9 + 574 z_1
      + 165 z_2 + 221 z_3 + 179 z_4
Subject To
 c1: x_0_0 = 6
 c2: x_0_1 + x_1_1 = 6
 c3: x_0_2 + x_1_2 + x_2_2 = 48
 c4: x_0_3 + x_1_3 + x_2_3 + x_3_3 = 5
 c5: x_0_4 + x_1_4 + x_2_4 + x_3_4 + x_4_4 = 15
 c6: x_0_5 + x_

In [11]:
pi = master.dual_values(contraintes5)
pi1 = contrainte4.dual_value
sc, sz = SP(pi, pi1)

Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 1116.085489 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 10 rows and 0 columns.
MIP Presolve modified 10 coefficients.
Reduced MIP has 10 rows, 20 columns, and 30 nonzeros.
Reduced MIP has 10 binaries, 10 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 10 rows, 20 columns, and 30 nonzeros.
Reduced MIP has 10 binaries, 10 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left   

In [55]:
Mc.append(sum(mtn_cost[t]*sz[t] for t in period_set))
C_t = np.vstack([C_t, sc])

z_5 = master.continuous_var(name = "z_5")

contrainte4.left_expr += z_5

for t in period_set:
    contraintes5[t].right_expr += sc[t]*z_5
    #print(contraintes5[t])

#print(Mc)
master.obj += Mc[4]*z_5
master.minimize(master.obj)

print(master.export_as_lp_string())
# Résolution
master_sol = master.solve(log_output = True)
objectives.append(master_sol.get_objective_value())
#master.print_information()
master.print_solution()

print(Mc)
print(objectives)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: clspm

Minimize
 obj: 7 x_0_0 + 11 x_0_1 + 11 x_0_2 + 16 x_0_3 + 16 x_0_4 + 17 x_0_5 + 18 x_0_6
      + 23 x_0_7 + 25 x_0_8 + 25 x_0_9 + 4 x_1_1 + 4 x_1_2 + 9 x_1_3 + 9 x_1_4
      + 10 x_1_5 + 11 x_1_6 + 16 x_1_7 + 18 x_1_8 + 18 x_1_9 + 7 x_2_2
      + 12 x_2_3 + 12 x_2_4 + 13 x_2_5 + 14 x_2_6 + 19 x_2_7 + 21 x_2_8
      + 21 x_2_9 + x_3_5 + 2 x_3_6 + 7 x_3_7 + 9 x_3_8 + 9 x_3_9 + 3 x_4_4
      + 4 x_4_5 + 5 x_4_6 + 10 x_4_7 + 12 x_4_8 + 12 x_4_9 + 9 x_5_5 + 10 x_5_6
      + 15 x_5_7 + 17 x_5_8 + 17 x_5_9 + 9 x_6_6 + 14 x_6_7 + 16 x_6_8
      + 16 x_6_9 + 9 x_7_7 + 11 x_7_8 + 11 x_7_9 + 2 x_8_8 + 2 x_8_9 + x_9_9
      + 9 y_1 + 4 y_2 + 8 y_3 + 4 y_4 + 6 y_5 + 10 y_6 + 9 y_8 + 7 y_9 + 574 z_1
      + 165 z_2 + 221 z_3 + 179 z_4 + 179 z_5
Subject To
 c1: x_0_0 = 6
 c2: x_0_1 + x_1_1 = 6
 c3: x_0_2 + x_1_2 + x_2_2 = 48
 c4: x_0_3 + x_1_3 + x_2_3 + x_3_3 = 5
 c5: x_0_4 + x_1_4 + x_2_4 + x_3_4 + x_4_4 = 15
 c6: 

In [56]:
pi = master.dual_values(contraintes5)
pi1 = contrainte4.dual_value
sc, sz = SP(pi, pi1)

Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 878.201206 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 10 rows and 0 columns.
MIP Presolve modified 10 coefficients.
Reduced MIP has 10 rows, 20 columns, and 30 nonzeros.
Reduced MIP has 10 binaries, 10 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 10 rows, 20 columns, and 30 nonzeros.
Reduced MIP has 10 binaries, 10 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left    

In [57]:
Mc.append(sum(mtn_cost[t]*sz[t] for t in period_set))
C_t = np.vstack([C_t, sc])

z_6 = master.continuous_var(name = "z_6")

contrainte4.left_expr += z_6

for t in period_set:
    contraintes5[t].right_expr += sc[t]*z_6
    #print(contraintes5[t])

#print(Mc)
master.obj += Mc[5]*z_6
master.minimize(master.obj)

print(master.export_as_lp_string())
# Résolution
master_sol = master.solve(log_output = True)
objectives.append(master_sol.get_objective_value())
#master.print_information()
master.print_solution()

print(Mc)
print(objectives)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: clspm

Minimize
 obj: 7 x_0_0 + 11 x_0_1 + 11 x_0_2 + 16 x_0_3 + 16 x_0_4 + 17 x_0_5 + 18 x_0_6
      + 23 x_0_7 + 25 x_0_8 + 25 x_0_9 + 4 x_1_1 + 4 x_1_2 + 9 x_1_3 + 9 x_1_4
      + 10 x_1_5 + 11 x_1_6 + 16 x_1_7 + 18 x_1_8 + 18 x_1_9 + 7 x_2_2
      + 12 x_2_3 + 12 x_2_4 + 13 x_2_5 + 14 x_2_6 + 19 x_2_7 + 21 x_2_8
      + 21 x_2_9 + x_3_5 + 2 x_3_6 + 7 x_3_7 + 9 x_3_8 + 9 x_3_9 + 3 x_4_4
      + 4 x_4_5 + 5 x_4_6 + 10 x_4_7 + 12 x_4_8 + 12 x_4_9 + 9 x_5_5 + 10 x_5_6
      + 15 x_5_7 + 17 x_5_8 + 17 x_5_9 + 9 x_6_6 + 14 x_6_7 + 16 x_6_8
      + 16 x_6_9 + 9 x_7_7 + 11 x_7_8 + 11 x_7_9 + 2 x_8_8 + 2 x_8_9 + x_9_9
      + 9 y_1 + 4 y_2 + 8 y_3 + 4 y_4 + 6 y_5 + 10 y_6 + 9 y_8 + 7 y_9 + 574 z_1
      + 165 z_2 + 221 z_3 + 179 z_4 + 179 z_5 + 207.000000000000 z_6
Subject To
 c1: x_0_0 = 6
 c2: x_0_1 + x_1_1 = 6
 c3: x_0_2 + x_1_2 + x_2_2 = 48
 c4: x_0_3 + x_1_3 + x_2_3 + x_3_3 = 5
 c5: x_0_4 + x_1_4 + x_2_4 + x

In [58]:

pricing = Model(name = "sous_probleme")
zt = pricing.binary_var_list(period_set, name = "z")
ct = pricing.integer_var_list(period_set, name = 'c')

def SP_MIP(pi: list):
    cost1 = sum(mtn_cost[t]*zt[t] + pi[t]*ct[t] for t in period_set)
    pricing.obj = cost1 - pi[-1]
    pricing.minimize(pricing.obj)
    
    ct1 = pricing.add_constraints(ct[t] <= cmax for t in period_set)
    ct2 = pricing.add_constraints(ct[t] <= alpha*ct[t-1] + cmax*zt[t] for t in period_set)

    # Résolution
    pricing_sol = pricing.solve(log_output = False)
    Z_SP = pricing_sol.get_objective_value()
    pricing.print_solution()
    sct = pricing_sol.get_values(ct)
    szt = pricing_sol.get_values(zt)
    Mc = sum(mtn_cost[t]*szt[t]for t in period_set)
    col = [sct,Mc]
    if Z_SP >=0 :
        print("RMP optimal")
        return(0)
    else:
        print("nouvelle colonne : ",szt)
        print("Z_SP =" ,Z_SP)
        return(col)
    

### Itération 1

In [45]:
c_init = [cmax for _ in period_set]
Mc = sum(mtn_cost)
objectives = []
#master
master = Model(name="clspm")
#x = master.integer_var_matrix(period_set, period_set, name = "Lot variable")
x = {(i,j): master.continuous_var(name=f'x_{i}_{j}') for i in range(T) for j in range(i, T)} #lot variable

y = master.continuous_var_list(period_set, name = "y") #set_up variable

c = master.continuous_var_list(period_set,name = "c") #capacity variable

z = master.continuous_var_list(1, 0.0, 1.0, name = 'z') #maintenance variable


contraintes1 = master.add_constraints(master.sum(x[k,t] for k in range(t+1)) == demand[t] for t in period_set)

contraintes2 = master.add_constraints(x[k,t] <= demand[t]*y[k] for k in period_set for t in range(k,T)) 

contraintes3 = master.add_constraints(master.sum(x[k,t] for t in range(k,T)) <= c[k] for k in period_set)

contrainte4 = master.add_constraint(z[0] == 1)

contraintes5 = master.add_constraints(c[t] == c_init[t]*z[0] for t in period_set)

#Fonction objectif
cost1 = sum(set_up_cost[k]*y[k] for k in period_set)
cost2 = sum(variable_prod_cost[k]*sum(x[k,t] for t in range(k,T)) for k in period_set)
cost3 = sum(x[k,t]*sum(inventory_cost[j] for j in range(k,t)) for k in period_set for t in range(k,T))
cost4 = Mc*z[0]
master.obj = cost1 + cost2 + cost3 + cost4
master.minimize(master.obj)

#print(master.export_as_lp_string())

# Résolution
master_sol = master.solve(log_output = False)
objective_value = master_sol.get_objective_value()
objectives.append(objective_value)
#master.print_information()
master.print_solution()

#Sauvegarde des variables duales 
pi = master.dual_values(contraintes5)
pi.append(contrainte4.dual_value)
print(pi)

objective: 1524.346
status: OPTIMAL_SOLUTION(2)
  x_0_0=6.000
  x_1_1=6.000
  x_1_2=35.000
  x_2_2=13.000
  x_3_3=5.000
  x_3_4=7.312
  x_3_5=10.237
  x_3_6=18.450
  x_4_4=7.688
  x_4_5=10.763
  x_4_6=22.550
  x_6_6=3.000
  x_7_7=18.000
  x_7_8=8.000
  x_8_8=41.000
  x_9_9=33.000
  y_0=1.000
  y_1=1.000
  y_2=0.271
  y_3=1.000
  y_4=0.513
  y_6=0.068
  y_7=1.000
  y_8=0.837
  y_9=1.000
  c_0=41.000
  c_1=41.000
  c_2=41.000
  c_3=41.000
  c_4=41.000
  c_5=41.000
  c_6=41.000
  c_7=41.000
  c_8=41.000
  c_9=41.000
  z_0=1.000
[0, -3.083333333333333, 0, -7.227272727272727, -4.177272727272727, 0, 0, 0, -8.816326530612244, 0, 0]


In [46]:
col=SP_MIP(pi)
print(col)

objective: -744.522
status: OPTIMAL_SOLUTION(2)
  z_1=1
  z_3=1
  z_8=1
  c_0=22
  c_1=41
  c_2=30
  c_3=41
  c_4=30
  c_5=22
  c_8=41
  c_9=30
nouvelle colonne :  [0, 1.0, 0, 1.0, 0, 0, 0, 0, 1.0, 0]
Z_SP = -744.5224180581323
[[22.0, 41.0, 30.0, 41.0, 30.0, 22.0, 0, 0, 41.0, 30.0], 165.0]


## Autres itérations 

In [47]:
i=2
stop=False
while(stop==False):
    print("\n")
    print("---------------- itération ",i, "----------------")
    pi=RMP(col)
   
    col=SP_MIP(pi)
    i+=1
    if col==0:
        stop=True
        
    
    
    



---------------- itération  2 ----------------
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: clspm

Minimize
 obj: 7 x_0_0 + 11 x_0_1 + 11 x_0_2 + 16 x_0_3 + 16 x_0_4 + 17 x_0_5 + 18 x_0_6
      + 23 x_0_7 + 25 x_0_8 + 25 x_0_9 + 4 x_1_1 + 4 x_1_2 + 9 x_1_3 + 9 x_1_4
      + 10 x_1_5 + 11 x_1_6 + 16 x_1_7 + 18 x_1_8 + 18 x_1_9 + 7 x_2_2
      + 12 x_2_3 + 12 x_2_4 + 13 x_2_5 + 14 x_2_6 + 19 x_2_7 + 21 x_2_8
      + 21 x_2_9 + x_3_5 + 2 x_3_6 + 7 x_3_7 + 9 x_3_8 + 9 x_3_9 + 3 x_4_4
      + 4 x_4_5 + 5 x_4_6 + 10 x_4_7 + 12 x_4_8 + 12 x_4_9 + 9 x_5_5 + 10 x_5_6
      + 15 x_5_7 + 17 x_5_8 + 17 x_5_9 + 9 x_6_6 + 14 x_6_7 + 16 x_6_8
      + 16 x_6_9 + 9 x_7_7 + 11 x_7_8 + 11 x_7_9 + 2 x_8_8 + 2 x_8_9 + x_9_9
      + 9 y_1 + 4 y_2 + 8 y_3 + 4 y_4 + 6 y_5 + 10 y_6 + 9 y_8 + 7 y_9 + 574 z_0
      + 165 z_1
Subject To
 c1: x_0_0 = 6
 c2: x_0_1 + x_1_1 = 6
 c3: x_0_2 + x_1_2 + x_2_2 = 48
 c4: x_0_3 + x_1_3 + x_2_3 + x_3_3 = 5
 c5: x_0_4 + x_1_4 + x_2_4 + x_3_4